# Week 1 - NLP and Deep Learning

---

Welcome to the weekly assignment of week 1. Note that the assignments are split up per lecture. 
Please upload your solutions on LearnIt as a notebook file to receive feedback.


# Lecture 1. What are words?


## 1. Regular Expressions
For this section, it might be handy to use the website https://regex101.com/ to test your solutions.

- a) Write a regular expression (regex or pattern) that matches any of the following words: `cat`, `sat`, `mat`.
<br>
(Bonus: What is a possible long solution? Can you find a shorter solution? *hint*: match characters instead of words)
- b) Write a regular expression that matches numbers, e.g. `12`, `1,000`, `39.95`.
- c) Expand the previous solution to match Danish price indications, e.g., `1,000 kr` or `39.95 DKK` or `19.95`.

## 2. Tokenization

**Note that we here use the old-fashioned meaning of tokenization which is the task of word segmentation (not to be confused with subword tokenization,which will be introduced in a later lecture**

(Adapted notebook from S. Riedel, UCL & Facebook: https://github.com/uclnlp/stat-nlp-book).

In Python, a simple way to tokenize a text is via the `split` method that divides a text wherever a particular substring is found. In the code below this pattern is simply the whitespace character, and this seems like a reasonable starting point for an English tokenization approach.

In [ ]:
text = """Mr. Bob Dobolina is thinkin' of a master plan.
Why doesn't he quit?"""
text.split(" ")

To make more fine-grained decisions, we will focus on using regular expressions for tokenization in this assignment. This can be done by either:
1. Defining the character sequence patterns at which to split.
2. Specifying patters that define what constitutes a token. 

In the code below we use a simple pattern `\s` that matches **any whitespace** to define where to split.

In [ ]:
import re
gap = re.compile(r'\s')
gap.split(text)

One **shortcoming** of this tokenization is its treatment of punctuation because it considers `plan.` as a token whereas ideally we would prefer `plan` and `.` to be distinct tokens. It might be easier to address this problem if we define what a token is, instead of what constitutes a gap. Below we have defined tokens as sequences of alphanumeric characters and punctuation.

In [ ]:
token = re.compile(r'\w+|[.?:]')
token.findall(text)

This still isn't perfect as `Mr.` is split into two tokens, but it should be a single token. Moreover, we have actually lost an apostrophe. Both are fixed below, although we now fail to break up the contraction `doesn't`.

In [ ]:
token = re.compile(r'Mr.|[\w\']+|[.?]')
tokens = token.findall(text)
tokens

In the code below, we have an input text and apply the tokenizer (described previously) on the text:

In [1]:
import re
text = """'Curiouser and curiouser!' cried Alice (she was so much surprised, that for the moment she quite
forgot how to speak good English); 'now I'm opening out like the largest telescope that ever was! Good-bye,
feet!' (for when she looked down at her feet, they seemed to be almost out of sight, they were getting so far
off). 'Oh, my poor little feet, I wonder who will put on your shoes and stockings for you now, dears? I'm sure I
shan't be able! I shall be a great deal too far off to trouble myself about you: you must manage the best
way you can; —but I must be kind to them,' thought Alice, 'or perhaps they won't walk the way I want to go!
Let me see: I'll give them a new pair of boots every Christmas...'
"""

token = re.compile(r'Mr.|[\w\']+|[.?]')
tokens = token.findall(text)
print(tokens[:10])
print(len(tokens))

["'Curiouser", 'and', 'curiouser', "'", 'cried', 'Alice', 'she', 'was', 'so', 'much']
147


Questions:

* a) The tokenizer clearly makes a few mistakes with punctuations. Where?

* b) Write a tokenizer to separate all tokenization characters as a single token. Note that there is no \p for punctuation in python `re` (so either make a list yourself, or you could specify it is any character which is not alphanumeric/whitespace).

* c) Should one separate `'m`, `'ll`, `n't`, possessives, and other forms of contractions from the word? Implement a tokenizer that separates these, and attaches the `'` to the latter part of the contraction (so that I'm -> I 'm and can't -> can 't).

* d) Should elipsis (...) be considered as three `.`s or one `...`? Design a regular expression that keeps the `.` together in 1 token.


## 3. Twitter Tokenization
As you might imagine, tokenizing tweets differs from standard tokenization. There are 'rules' on what specific elements of a tweet might be (mentions, hashtags, links), and how they are tokenized. The goal of this exercise is not to create a bullet-proof Twitter tokenizer but to understand tokenization in a different domain.

In the next exercises, we will focus on the following tweet:

In [ ]:
tweet = "@robv New vids coming tomorrow #excited_as_a_child, can't w8!!"

In [ ]:
token = re.compile(r'[\w]+')
tokens = token.findall(tweet)
print(tokens)

Questions:
- a) What is the correct tokenization of the tweet above according to you?
- b) Try your tokenizer from the previous exercise (Question 2). Which cases are going wrong? Rewrite your tokenizer such that it handles the above tweet correctly.
- c) How will your tokenizer handle emojis?
- d) Think of at least one example where your tokenizer (from b) will behave incorrectly.

## 4. Segmentation


Sentence segmentation is not a trivial task either.

First, make sure you understand the following sentence segmentation code:

In [3]:
import re

def sentence_segment(match_regex, tokens):
    """
    Splits a sequence of tokens into sentences, splitting wherever the given matching regular expression
    matches.

    Parameters
    ----------
    tokens      the input sequence as list of strings (each item is a ``word'')
    match_regex the regular expression that defines at which token to split.

    Returns
    -------
    a list of lists of strings, where each string is a word, and each inner list
    represents a sentence.
    """
    sentences = [[]]
    for tok in tokens:
        sentences[-1].append(tok)
        if match_regex.match(tok):
            sentences.append([])
            
    if sentences[-1] == []:
        del sentences[-1]
    return sentences


In the following code, there is a variable `text` containing a small text and a regular expression-based segmenter:

In [4]:
text = """
Llanfairpwllgwyngyllgogerychwyrndrobwllllantysiliogogogoch is the longest official one-word placename in U.K. Isn't that weird? I mean, someone took the effort to really make this name as complicated as possible, huh?! Of course, U.S.A. also has its own record in the longest name, albeit a bit shorter... This record belongs to the place called Chargoggagoggmanchauggagoggchaubunagungamaugg. There's so many wonderful little details one can find out while browsing http://www.wikipedia.org during their Ph.D. or an M.Sc.
"""

token = re.compile(r'Mr.|[\w\']+|[.?]+')

tokens = token.findall(text)
sentences = sentence_segment(re.compile(r'\.'), tokens)

for sentence in sentences:
    print(sentence)

['Llanfairpwllgwyngyllgogerychwyrndrobwllllantysiliogogogoch', 'is', 'the', 'longest', 'official', 'one', 'word', 'placename', 'in', 'U', '.']
['K', '.']
["Isn't", 'that', 'weird', '?', 'I', 'mean', 'someone', 'took', 'the', 'effort', 'to', 'really', 'make', 'this', 'name', 'as', 'complicated', 'as', 'possible', 'huh', '?', 'Of', 'course', 'U', '.']
['S', '.']
['A', '.']
['also', 'has', 'its', 'own', 'record', 'in', 'the', 'longest', 'name', 'albeit', 'a', 'bit', 'shorter', '...']
['This', 'record', 'belongs', 'to', 'the', 'place', 'called', 'Chargoggagoggmanchauggagoggchaubunagungamaugg', '.']
["There's", 'so', 'many', 'wonderful', 'little', 'details', 'one', 'can', 'find', 'out', 'while', 'browsing', 'http', 'www', '.']
['wikipedia', '.']
['org', 'during', 'their', 'Ph', '.']
['D', '.']
['or', 'an', 'M', '.']
['Sc', '.']


Questions:
- a) Improve the segmenter so that it handles a list of abbreviations (U.S./U.K./Ph.D/M.Sc) correctly.
- b) Improve the segmenter so that it handles URL's correctly. You can assume that URLs start with "www."

## 5. Tokenization competition

Tokenization of social media can be more challenging. We provide a small development set for you to experiment with, which you can find in `week1/tok.dev.txt`. The file is a tab-separated file, where the first column contains the input text, and the second column the gold tokenization (as decided by an annotator).

In [ ]:
data = [line.strip().split('\t') for line in open('tok.dev.txt', encoding="utf-8")]
print(data[0])

There is also a test file with the same format, but the gold annotation is missing. This can be found in `week1/tok.test.txt`. You are supposed to develop your tokenizer based on the development data, and then apply your tokenizer on the test data. You can hand in the predictions on the test data on LearnIt in the same slot as the rest of the assignment. We will use F1 score for evaluation.

Make sure that the file you hand in:
- Contains only the output of your system, so it should have the same number of lines as the test data file, but contains different placements of whitespace characters (the characters without whitespaces should be exactly the same though). 
- Has your ITU username as the name of the file: i.e. `robv.txt`.
- You can test whether the format matches by using the evaluation script provided (described below) on the development data.

We have provided an evaluation script for your convenience, it returns F1 score, recall, and precision. It also prints out all sentences where your model made an error (indicating the error in red if supported by your terminal), and checks whether your output is in the right format. It can be found in `week1/tok_eval.py`.

# Lecture 2: Language (correction)

## 6. Spelling correction

Below is an implementation of the Levenshtein distance. It uses a some efficiency tricks, and it is not important you understand every line of this implementation.

In [ ]:
def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

print(levenshteinDistance('this', 'that'))

We also provide you with an English word list from [Aspell](http://aspell.net/) in `aspell-en-dict.txt`. It can be used as follows:

In [ ]:
# Load wordlist (one word per line)
en_dict = set([word.strip() for word in open('aspell-en-dict.txt', encoding="utf-8").readlines()])
    
# Example usage
typo = 'brower'
correction = 'browser'
print(typo, correction)
print(typo in en_dict)
print(correction in en_dict)
print(levenshteinDistance(typo, correction))

* a) Implement a (naive) spelling correction system that finds the word in the word list with the smallest minimum edit distance for a word that contains a misspelling. 
* b) There could be multiple words with the smallest minimum edit distance for some typos, what are supplementary methods to re-rank these? (mention at least 2)

## 7. Evaluation and Analysis of spelling correction

We also provide you with a list of 100 typos and their corrections from the [GitHub Typo Corpus](https://aclanthology.org/2020.lrec-1.835/) in `typos.txt`. It can be used as follows:

In [ ]:
# Load github typo corpus misspellings
typos = []
corrections = []
for line in open('typos.txt', encoding="utf-8"):
    tok = line.strip().split('\t')
    typos.append(tok[0])
    corrections.append(tok[1])
    
# Example usage
print(typos[0], corrections[0])

* a) Evaluate the spelling correction system you implemented in the previous assignment with accuracy. How many of the words did it correct right?
* b) Now evaluate the errors, can you identify some common causes (i.e. trends) in the mistakes of your model?

## 8. Levenshtein distance in practice
Work out the distance matrix of the edit distance for the two words: `dansk` and `dane`. This is not a programming assignment, but rather a pen-and-paper assignment. You can fill out a table in any editor you like, or draw the matrix on a piece of paper and upload a picture or scan. An example of such a matrix was shown in class, but is also included in the Speech and Language Processing book (figure 2.18).